# CI-CD Pipeline

In [26]:
# Import libraries
import sys
import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile
from sagemaker.model import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.transformer import Transformer
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import Join
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.pipeline import Pipeline
from pprint import pprint
import time
import json
from sagemaker.lineage.visualizer import LineageTableVisualizer

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
pipeline_session = PipelineSession()
bucket = sagemaker_session.default_bucket()

base_dir = "/opt/ml/processing"
instance_type = "ml.m5.xlarge"
model_package_group_name = "NutriscoreModelPackageGroupName"

In [2]:
# Set dataset S3 paths
input_data_uri = f"s3://{bucket}/food_us_subset_100k/"
batch_data_uri = f"s3://{bucket}/nutriscore-prediction-xgboost/test/test_features_only.csv"

## Define Parameters to Parametrize Pipeline Execution

In [3]:
# Define parameters
processing_instance_count = ParameterInteger(name="ProcessingInstanceCount", default_value=1)
instance_type = ParameterString(name="TrainingInstanceType", default_value=instance_type)
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)
batch_data = ParameterString(
    name="BatchData",
    default_value=batch_data_uri
)
rmse_threshold = ParameterFloat(name="RmseThreshold", default_value=2.0)

In [4]:
# Define processing step
sklearn_processor = SKLearnProcessor(
    framework_version="1.2-1",
    instance_type=instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-nutriscore-process",
    role=role,
    sagemaker_session=pipeline_session,
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [5]:
# Run the preprocessing script
processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(source=input_data, destination=f"{base_dir}/input"),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source=f"{base_dir}/train"),
        ProcessingOutput(output_name="validation", source=f"{base_dir}/validation"),
        ProcessingOutput(output_name="test", source=f"{base_dir}/test"),
    ],
    code="utils/preprocess_nutriscore.py",
    arguments=["--input-path", "/opt/ml/processing/input"],
)

step_process = ProcessingStep(name="NutriscoreProcess", step_args=processor_args)

/opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Define a Training Step to Train a Model

In [6]:
# Define model training step
model_path = f"s3://{bucket}/NutriscoreTrain"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.7-1",
    instance_type=instance_type,
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
    sagemaker_session=pipeline_session,
)
xgb_train.set_hyperparameters(
    objective="reg:squarederror",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

train_args = xgb_train.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)
step_train = TrainingStep(
    name="NutriscoreTrain",
    step_args=train_args,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.


## Define a Model Evaluation Step

In [7]:
# Define model evaluator
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=instance_type,
    instance_count=1,
    base_job_name="script-nutriscore-eval",
    role=role,
    sagemaker_session=pipeline_session,
)

eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="utils/evaluate_nutriscore.py",
)

In [8]:
# Define evaluation step
evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)
step_eval = ProcessingStep(
    name="NutriscoreEval",
    step_args=eval_args,
    property_files=[evaluation_report],
)

## Define a Create Model Step to Create a Model

In [9]:
# Define model step
model = Model(
    image_uri=image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_model = ModelStep(
    name="NutriscoreCreateModel",
    step_args=model.create(instance_type=instance_type, accelerator_type="ml.eia1.medium"),
)

## Define a Transform Step to Perform Batch Transformation

In [16]:
# Define batch transform step
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type=instance_type,
    instance_count=1,
    output_path=f"s3://{bucket}/NutriscoreTransform",
)

step_transform = TransformStep(
    name="NutriscoreTransform", 
    transformer=transformer, 
    inputs=TransformInput(data=batch_data, content_type="text/csv")
)

## Define a Register Model Step to Create a Model Package

In [17]:
# Define register model step
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", instance_type],
    transform_instances=[instance_type],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)
step_register = ModelStep(name="NutriscoreRegisterModel", step_args=register_args)

/opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Define a Fail Step to Terminate the Pipeline Execution and Mark it as Failed

In [18]:
# Define fail step
step_fail = FailStep(
    name="NutriscoreRMSEFail",
    error_message=Join(on=" ", values=["Execution failed due to RMSE >", rmse_threshold]),
)

## Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry, Or Terminate the Execution in Failed State

In [19]:
# Define condition step
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.rmse.value",
    ),
    right=rmse_threshold,
)

step_cond = ConditionStep(
    name="NutriscoreRMSECond",
    conditions=[cond_lte],
    if_steps=[step_register, step_create_model, step_transform],
    else_steps=[step_fail],
)

## Define a Pipeline of Parameters, Steps, and Conditions

In [20]:
# Define pipeline
pipeline_name = f"NutriscorePipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        instance_type,
        model_approval_status,
        input_data,
        batch_data,
        rmse_threshold,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
)

## Submit Pipeline to SageMaker and Start Execution

In [21]:
# Submit pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()

In [22]:
# Describe the pipeline execution
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:654654380268:pipeline/NutriscorePipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:654654380268:pipeline/NutriscorePipeline/execution/e8gtivnsjkdj',
 'PipelineExecutionDisplayName': 'execution-1760646089866',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'nutriscorepipeline',
  'TrialName': 'e8gtivnsjkdj'},
 'CreationTime': datetime.datetime(2025, 10, 16, 20, 21, 29, 808000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 10, 16, 20, 35, 5, 849000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:654654380268:user-profile/d-bzkrqbjrtwzf/amalinsky',
  'UserProfileName': 'amalinsky',
  'DomainId': 'd-bzkrqbjrtwzf',
  'IamIdentity': {'Arn': 'arn:aws:sts::654654380268:assumed-role/LabRole/SageMaker',
   'PrincipalId': 'AROAZQ3DRDDWK3JDJSTXQ:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:654654380268:user-prof

In [23]:
# Wait for execution to complete (around 15 minutes)
execution.wait()

In [24]:
# List execution steps
execution.list_steps()

[{'StepName': 'NutriscoreTransform',
  'StartTime': datetime.datetime(2025, 10, 16, 20, 29, 16, 674000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 10, 16, 20, 35, 5, 673000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:us-east-1:654654380268:transform-job/pipelines-e8gtivnsjkdj-NutriscoreTransform-uFSIJtkFJo'}},
  'AttemptCount': 1},
 {'StepName': 'NutriscoreCreateModel-CreateModel',
  'StartTime': datetime.datetime(2025, 10, 16, 20, 29, 14, 411000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 10, 16, 20, 29, 16, 182000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:654654380268:model/pipelines-e8gtivnsjkdj-NutriscoreCreateMode-EU6329lfnc'}},
  'AttemptCount': 1},
 {'StepName': 'NutriscoreRegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2025, 10, 16, 20, 29, 14, 411000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 1

## Examine the Evaluation

In [27]:
evaluation_json = sagemaker.s3.S3Downloader.read_file(
    "{}/evaluation.json".format(
        step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    )
)
pprint(json.loads(evaluation_json))

{'regression_metrics': {'rmse': {'standard_deviation': 1.806670787091679,
                                 'value': 1.806671120100456}}}


## Review Artifact Lineage

In [28]:
viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

{'StepName': 'NutriscoreProcess', 'StartTime': datetime.datetime(2025, 10, 16, 20, 21, 30, 674000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2025, 10, 16, 20, 24, 3, 358000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:654654380268:processing-job/pipelines-e8gtivnsjkdj-NutriscoreProcess-pWNq13lERk'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...c4b10cbd6321ea5/preprocess_nutriscore.py,Input,DataSet,ContributedTo,artifact
1,s3://...east-1-654654380268/food_us_subset_100k/,Input,DataSet,ContributedTo,artifact
2,68331...com/sagemaker-scikit-learn:1.2-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...gtivnsjkdj/NutriscoreProcess/output/test,Output,DataSet,Produced,artifact
4,s3://...jkdj/NutriscoreProcess/output/validation,Output,DataSet,Produced,artifact
5,s3://...tivnsjkdj/NutriscoreProcess/output/train,Output,DataSet,Produced,artifact


{'StepName': 'NutriscoreTrain', 'StartTime': datetime.datetime(2025, 10, 16, 20, 24, 4, 142000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2025, 10, 16, 20, 26, 37, 779000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:654654380268:training-job/pipelines-e8gtivnsjkdj-NutriscoreTrain-1LuYGQypPQ'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...jkdj/NutriscoreProcess/output/validation,Input,DataSet,ContributedTo,artifact
1,s3://...tivnsjkdj/NutriscoreProcess/output/train,Input,DataSet,ContributedTo,artifact
2,68331...-1.amazonaws.com/sagemaker-xgboost:1.7-1,Input,Image,ContributedTo,artifact
3,s3://...coreTrain-1LuYGQypPQ/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'NutriscoreEval', 'StartTime': datetime.datetime(2025, 10, 16, 20, 26, 38, 698000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2025, 10, 16, 20, 29, 12, 745000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:654654380268:processing-job/pipelines-e8gtivnsjkdj-NutriscoreEval-pc8xRTHKzG'}}, 'AttemptCount': 1}


...

,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...coreTrain-1LuYGQypPQ/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,68331...-1.amazonaws.com/sagemaker-xgboost:1.7-1,Input,Image,ContributedTo,artifact
2,NutriscoreModelPackageGroupName-2-1760646555-a...,Input,ModelLifeCycle,ContributedTo,action
3,NutriscoreModelPackageGroupName-2-PendingManua...,Input,Approval,ContributedTo,action
4,NutriscoreModelPackageGroupName-1760594124-aws...,Output,ModelGroup,AssociatedWith,context


{'StepName': 'NutriscoreCreateModel-CreateModel', 'StartTime': datetime.datetime(2025, 10, 16, 20, 29, 14, 411000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2025, 10, 16, 20, 29, 16, 182000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:us-east-1:654654380268:model/pipelines-e8gtivnsjkdj-NutriscoreCreateMode-EU6329lfnc'}}, 'AttemptCount': 1}


None

{'StepName': 'NutriscoreTransform', 'StartTime': datetime.datetime(2025, 10, 16, 20, 29, 16, 674000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2025, 10, 16, 20, 35, 5, 673000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TransformJob': {'Arn': 'arn:aws:sagemaker:us-east-1:654654380268:transform-job/pipelines-e8gtivnsjkdj-NutriscoreTransform-uFSIJtkFJo'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...coreTrain-1LuYGQypPQ/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,68331...-1.amazonaws.com/sagemaker-xgboost:1.7-1,Input,Image,ContributedTo,artifact
2,s3://...tion-xgboost/test/test_features_only.csv,Input,DataSet,ContributedTo,artifact
3,s3://...-east-1-654654380268/NutriscoreTransform,Output,DataSet,Produced,artifact


![Pipeline-SuccessState](img/Pipeline-SuccessState.png)

## Rerun Pipeline to Trigger Fail Case

In [29]:
# Set RMSE threshold lower to trigger fail state
# 'Execution failed due to RMSE > 1.0'
execution = pipeline.start(parameters=dict(RmseThreshold=1.0))

In [30]:
try:
    execution.wait()
except Exception as error:
    print(error)

Waiter PipelineExecutionComplete failed: Waiter encountered a terminal failure state: For expression "PipelineExecutionStatus" we matched expected path: "Failed"


In [31]:
execution.list_steps()

[{'StepName': 'NutriscoreRMSEFail',
  'StartTime': datetime.datetime(2025, 10, 16, 21, 7, 26, 870000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 10, 16, 21, 7, 27, 417000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': 'Execution failed due to RMSE > 1.0',
  'Metadata': {'Fail': {'ErrorMessage': 'Execution failed due to RMSE > 1.0'}},
  'AttemptCount': 1},
 {'StepName': 'NutriscoreRMSECond',
  'StartTime': datetime.datetime(2025, 10, 16, 21, 7, 25, 814000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 10, 16, 21, 7, 26, 159000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Condition': {'Outcome': 'False'}},
  'AttemptCount': 1},
 {'StepName': 'NutriscoreEval',
  'StartTime': datetime.datetime(2025, 10, 16, 21, 2, 21, 411000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 10, 16, 21, 7, 25, 151000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:6546543

![Pipeline-FailState](img/Pipeline-FailState.png)